## 10.4 Shoppy

W tej apllikacji zaimplementujemy architekturę `MVVM`. Aplikacja będzie listą zakupową z bazą `ROOM` umożliwiająca dodanie elementu, usunięcie, edycję oraz wyszukanie. Wykorzystamy `Jetpack Navigation`do nawigowania po trzech fragmentach
- `ListFragment` - fragment domowy wyświetlający listę wszystkich elementów oraz pozwalający przeszukać bazę
- `AddFragment` - przejście z `ListFragment` za pomocą `FAB` - umożliwia dodanie nowego elementu do bazy
- `UpdateFragment` - przejście przez kliknięcie elementu `RecyclerView` na `ListFragment` - umożliwia edycję wybranego elementu.

Dodajmy wymagane zależności do plików `build.gradle`

In [ ]:
// Project
buildscript { // przed blokiem plugins
    repositories {
        google()
    }
    dependencies {
        classpath "androidx.navigation:navigation-safe-args-gradle-plugin:2.5.1"
    }
}

In [ ]:
// Module
plugins {
    id 'com.android.application'
    id 'org.jetbrains.kotlin.android'
    id 'androidx.navigation.safeargs.kotlin'
    id 'kotlin-android'
    id 'kotlin-kapt'
}
...
buildFeatures {
    viewBinding true
}
...
dependencies {

    // ROOM
    implementation("androidx.room:room-ktx:2.4.3")
    kapt("androidx.room:room-compiler:2.4.3")

    // ViewModel
    implementation 'androidx.lifecycle:lifecycle-viewmodel-ktx:2.5.1'
    // LiveData
    implementation "androidx.lifecycle:lifecycle-livedata-ktx:2.5.1"

    // Fragment
    implementation "androidx.fragment:fragment-ktx:1.5.2"

    // Navigation
    implementation "androidx.navigation:navigation-fragment:2.5.1"
    implementation "androidx.navigation:navigation-ui:2.5.1"
    ...
}

### **data**

Rozpocznijmy od zdefiniowania modelu - klasa `Item` będzie posiadała trzy pola
- `id` - unikalny identyfikator
- `name` - nazwa produktu
- `quantity` - ilość danego produktu
Nasza klasa będzie reprezentowała tabelę w bazie `ROOM`, więc dodamy również adnotację `@Entity`

In [ ]:
@Entity(tableName = "item_table")
data class Item (
    @PrimaryKey(autoGenerate = true)
    val id: Int,
    val name: String,
    val quantity: Int
)

Wykorzystujemy pole `id` jako `PrimaryKey` który będzie automatycznie generowany przez `ROOM`.

Zdefiniujmy nasz `DAO` z odpowiednimi metodami

In [ ]:
interface ItemDao {

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    suspend fun addItem(item: Item) // dodaj element

    @Query("SELECT * FROM item_table ORDER BY id ASC")
    fun readAllData(): LiveData<List<Item>> // czytaj wszystkie elementy

    @Query("SELECT * FROM item_table WHERE id = :id")
    fun getItem(id: Int): LiveData<Item> // czytaj element o zadanym id

    @Update(onConflict = OnConflictStrategy.REPLACE)
    suspend fun updateItem(item: Item) // aktualizuj element

    @Delete
    suspend fun deleteItem(item: Item) // usuń element

    @Query("DELETE FROM item_table")
    suspend fun deleteAllItems() // usuń wszystkie elementy

    @Query("SELECT * FROM item_table WHERE name LIKE :query")
    fun search(query: String): LiveData<List<Item>> // wyszukaj element o zadanej nazwie
}

Kolejnym elementem będzie baza danych `ROOM` - tak jak poprzednio wykorzystamy `ExecutorService` to zapisu do bazy. Z bazy dane zwrócimy jako `LiveData`

In [ ]:
@Database(entities = [Item::class], version = 1, exportSchema = false)
abstract class ItemDatabase : RoomDatabase() {

    abstract fun itemDao(): ItemDao

    companion object{
        @Volatile private var INSTANCE: ItemDatabase? = null

        fun getDatabase(context: Context): ItemDatabase {
            return INSTANCE ?: synchronized(this) {
                val instance = Room.databaseBuilder(
                    context.applicationContext,
                    ItemDatabase::class.java,
                    "word_database_kotlin"
                ).build().also { INSTANCE = it }
                instance
            }
        }
    }
}

Następnie zdefiniujmy `Repository` - element nie należący do architektury `MVVM` lecz mocno zalecany. Dodajmy dwa pola `ItemDao` oraz `readAllData`

In [ ]:
class ItemRepository(private val itemDao: ItemDao) {
    val readAllData: LiveData<List<Item>> = itemDao.readAllData()

W bloku `init` inicjujemy `itemDao`

In [ ]:
    init {
        val itemDao = ItemDatabase.getDatabase(application).itemDao()
    }

I wystawiamy wszystkie metody interfejsu `ItemDao`

In [ ]:
    suspend fun addItem(item: Item){
        itemDao.addItem(item)
    }

    fun getItem(id: Int): LiveData<Item>{
        return itemDao.getItem(id)
    }

    suspend fun updateItem(item: Item){
        itemDao.updateItem(item)
    }

    suspend fun deleteItem(item: Item){
        itemDao.deleteItem(item)
    }

    suspend fun deleteAllItems(){
        itemDao.deleteAllItems()
    }

    fun searchItem(query: String): LiveData<List<Item>>{
        return itemDao.search(query)
    }

Ostatnim elementem danych będzie `ItemViewModel` - dodajmy dwa pola przechowujące `Repository` oraz listę wszystkich elementów 

In [ ]:
class ItemViewModel(application: Application) : AndroidViewModel(application) {
    val readAllData: LiveData<List<Item>>
    private val repository: ItemRepository

W bloku `init` zainicjujmy te dwa pola

In [ ]:
    init {
        repository = ItemRepository(itemDao)
        readAllData = repository.readAllData
    }

Dodajmy metody odczytujące dane z bazy i zwracające `LiveData`

In [ ]:
    fun getItem(id: Int): LiveData<Item>{
        return repository.getItem(id)
    }

    fun searchItem(query: String): LiveData<List<Item>>{
        return repository.searchItem(query)
    }

oraz metody zapisujące dane do bazy - tutaj wykorzystamy `ExecutorService` zdefiniowany w klasie `ItemDatabase`

In [ ]:
    fun updateItem(item: Item){
        viewModelScope.launch {
            repository.updateItem(item)
        }
    }

    fun deleteItem(item: Item){
        viewModelScope.launch {
            repository.deleteItem(item)
        }
    }

    fun deleteAll(){
        viewModelScope.launch {
            repository.deleteAllItems()
        }
    }
    
    fun addItem(item: Item){
        viewModelScope.launch {
            repository.addItem(item)
        }
    }

### **nawigacja**

Do layoutu aktywności dodajemy `FragmentContainerView`

In [ ]:
<androidx.constraintlayout.widget.ConstraintLayout 
    xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".MainActivity">

    <androidx.fragment.app.FragmentContainerView
        android:id="@+id/fragmentContainerView"
        android:name="androidx.navigation.fragment.NavHostFragment"
        android:layout_width="409dp"
        android:layout_height="match_parent"
        app:defaultNavHost="true"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintHorizontal_bias="0.5"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent"
        app:navGraph="@navigation/navigation" />
</androidx.constraintlayout.widget.ConstraintLayout>

Layouty `AddFragment` oraz `UpdateFragment` będą zawierać dwa pola `EditText` oraz przycisk dzięki któremu wykonamy zapis/aktualizację

In [ ]:
<androidx.constraintlayout.widget.ConstraintLayout 
    xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".fragments.add.AddFragment">

    <EditText
        android:id="@+id/nameEditText"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:hint="@string/nazwa"
        android:textSize="24sp"
        android:gravity="center_horizontal"
        android:layout_marginStart="16dp"
        android:layout_marginTop="50dp"
        android:layout_marginEnd="16dp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent"
        android:inputType="text"
        android:importantForAutofill="no" />

    <EditText
        android:id="@+id/quantityEditText"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginStart="16dp"
        android:layout_marginTop="16dp"
        android:layout_marginEnd="16dp"
        android:gravity="center_horizontal"
        android:hint="@string/ilosc"
        android:importantForAutofill="no"
        android:textSize="24sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/nameEditText"
        android:inputType="number" />

    <Button
        android:id="@+id/saveButton"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginStart="24dp"
        android:layout_marginTop="16dp"
        android:layout_marginEnd="24dp"
        android:text="@string/dodaj"
        android:textSize="24sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/quantityEditText" />


</androidx.constraintlayout.widget.ConstraintLayout>

Layout `ListFragment` będzie zawiera `RecyclerView`, dwa `FloatingActionButton` (przejście do `AddFragment` oraz czyszczenie całej listy), oraz pole `SearchView`, które wykorzystamy do przeszukiwania bazy.

Dodajmy nawigację, do `ListFragment` dodajemy dwie akcje
- przejście do `AddFragment`
- przejście do `UpdateFragment` - tutaj przekazujemy `id`

In [ ]:
    <fragment
        android:id="@+id/listFragment"
        android:name="pl.udu.uwr.pum.shoppyjava.fragments.list.ListFragment"
        android:label="@string/lista"
        tools:layout="@layout/fragment_list" >
        <action
            android:id="@+id/action_listFragment_to_addFragment"
            app:destination="@id/addFragment" />
        <action
            android:id="@+id/action_listFragment_to_updateFragment"
            app:destination="@id/updateFragment" >
            <argument
                android:name="id"
                app:argType="integer" />
        </action>
    </fragment>

W pozostałych dwóch fragmentach definiujemy akcję przejścia powrotnego na `ListFragment`

In [ ]:
    <fragment
        android:id="@+id/addFragment"
        android:name="pl.udu.uwr.pum.shoppyjava.fragments.add.AddFragment"
        android:label="@string/dodaj"
        tools:layout="@layout/fragment_add" >
        <action
            android:id="@+id/action_addFragment_to_listFragment"
            app:destination="@id/listFragment" />
    </fragment>
    <fragment
        android:id="@+id/updateFragment"
        android:name="pl.udu.uwr.pum.shoppyjava.fragments.update.UpdateFragment"
        android:label="@string/edytuj"
        tools:layout="@layout/fragment_update" >
        <action
            android:id="@+id/action_updateFragment_to_listFragment"
            app:destination="@id/listFragment" />
    </fragment>

### **MainActivity**

W głównej aktywności umożliwimy nawigację wsteczną z poziomu `ActionBar`

In [ ]:
class MainActivity : AppCompatActivity() {

    private val binding: ActivityMainBinding by lazy {
        ActivityMainBinding.inflate(layoutInflater)
    }


    private val navController: NavController by lazy {
        val navHostFragment = 
        supportFragmentManager.findFragmentById(R.id.fragmentContainerView)
                as NavHostFragment
        navHostFragment.findNavController()
    }

    private val appBarConfiguration: AppBarConfiguration by lazy {
        AppBarConfiguration(navController.graph)
    }

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(binding.root)

        setupActionBarWithNavController(navController, appBarConfiguration)
    }

    override fun onSupportNavigateUp(): Boolean {
        return navController.navigateUp(appBarConfiguration)
                || super.onSupportNavigateUp()
    }
}

### **AddFragment**

Rozpocznijmy od dodania `ItemViewModel`

In [ ]:
    private val itemViewModel: ItemViewModel by viewModels()

Zdefiniujmy metode `insertToDatabase` - przyjmuje ona jako argument `View`, który jest wymagany przy wywołaniu metody `findNavController`

In [ ]:
    private fun insertToDatabase() {

W pierwszym kroku sprawdźmy czy pole `EditText` nie są puste

In [ ]:
    val name = binding.nameEditText.text.toString()
    val quantity = binding.quantityEditText.text.toString()

    if (name.isNotEmpty() && quantity.isNotEmpty()){

Jeżeli tak jest - tworzymy nowy `Item` z dostępnych danych i wykonujemy metodę `insert` z klasy `ItemViewModel`

In [ ]:
    if (name.isNotEmpty() && quantity.isNotEmpty()){
        val item = Item(0, name, quantity.toInt())
        itemViewModel.addItem(item)
        findNavController()
            .navigate(
                AddFragmentDirections.actionAddFragmentToListFragment())
    } 

Ostatnim elementem jest powrót do `ListFragemnt` po dodaniu nowego elementu

In [ ]:
findNavController()
    .navigate(AddFragmentDirections.actionAddFragmentToListFragment())

W przeciwnym wypadku wyświetlamy błąd

In [ ]:
    } else{
        binding.nameEditText.error = "Podaj nazwę"
        binding.quantityEditText.error = "Podaj ilość"
    }

W metodzie `onViewCreated` dodajemy obsługę `onClick` przycisku odpowiadającego za aktualizację

In [ ]:
binding.saveButton.setOnClickListener {insertToDatabase()}

### **UpdateFragment**

Rozpocznijmy od dodania pola `ItemViewModel` oraz `int` reprezentujący odebrane `id` z `ListFragment`

In [ ]:
private val itemViewModel: ItemViewModel by viewModels()

private val itemId: Int by lazy { requireArguments().getInt("id") }

Zdefiniujmy metodę wypełniającą pola `EditText` danymi pochodziącymi z elementu odpowiadającego `id` przekazanemu w arggumentach.

In [ ]:
private fun displayItem(item: Item){
    binding.nameEditText.setText(item.name)
    binding.quantityEditText.setText(item.quantity.toString())
}

Dodajmy również metodę `updateItem`, która jest niemal identyczna jak poprzednio zdefiniowana `insertToDatabase`

In [ ]:
    private fun updateItem() {
        val name = binding.nameEditText.text.toString()
        val quantity = binding.quantityEditText.text.toString()

        if (name.isNotEmpty() && quantity.isNotEmpty()){
            val item = Item(itemId, name, quantity.toInt())
            itemViewModel.updateItem(item)
            findNavController()
                .navigate(UpdateFragmentDirections
                        .actionUpdateFragmentToListFragment())
        } else{
            binding.nameEditText.error = "Podaj nazwę"
            binding.quantityEditText.error = "Podaj ilość"
        }
    }

W metodzie `onViewCreated` wyciągamy element z bazy za pomocą metody `getItem` i dodajemy obserwator - `displayItem`

In [ ]:
itemViewModel.getItem(itemId)
    .observe(viewLifecycleOwner, this::displayItem)

oraz zaimplementujmy metodę `onClick` przycisku aktualizującego element

In [ ]:
binding.updateButton.setOnClickListener { updateItem() }

### **ListFragment**

Rozpocznijmy implementację od zdefiniowania layoutu pojedynczego elementu

In [ ]:
<androidx.constraintlayout.widget.ConstraintLayout 
    xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:padding="24dp">

    <TextView
        android:id="@+id/nameTextViewRV"
        android:layout_width="0dp"
        android:layout_height="wrap_content"
        android:text="nazwa przedmiotu"
        android:textSize="24sp"
        android:textStyle="bold"
        app:layout_constraintEnd_toStartOf="@+id/guideline"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/quantityTextViewRV"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:gravity="end"
        android:text="999"
        android:textSize="24sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="@+id/guideline"
        app:layout_constraintTop_toTopOf="parent" />

    <androidx.constraintlayout.widget.Guideline
        android:id="@+id/guideline"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:orientation="vertical"
        app:layout_constraintGuide_percent="0.75" />


</androidx.constraintlayout.widget.ConstraintLayout>

Dodajmy `ItemViewHolder`

In [ ]:
class ItemViewHolder(private val binding: ItemRecyclerviewBinding) 
    : RecyclerView.ViewHolder(binding.root) {

    fun bind(item: Item) {
        binding.nameTextViewRV.text = item.name
        binding.quantityTextViewRV.text = item.quantity.toString()

        binding.root.setOnClickListener {
            val action: NavDirections = ListFragmentDirections
                .actionListFragmentToUpdateFragment(item.id)
            findNavController(binding.root).navigate(action)
        }
    }
}

Dodajemy implementację `onClick` elementu listy - przechodzimy do `UpdateFragment` przekazując `id` klikniętego elementu.

Dodajmy `ItemComparator` - dzięki niemu możemy zdefiniować metody porównujące elementy - jest wykorzystywany podczas wywołania `submitList` w klasie `Listfragment` (niejawnie)

In [ ]:
class ItemComparator : DiffUtil.ItemCallback<Item>() {
    override fun areItemsTheSame(oldItem: Item, newItem: Item): Boolean {
        return oldItem === newItem
    }

    override fun areContentsTheSame(oldItem: Item, newItem: Item): Boolean {
        return oldItem.name == newItem.name && oldItem.quantity == newItem.quantity
    }
}

Klasa `ItemAdapter` będzie rozszerzać `ListAdapter` - co ułatwi indeksowanie i zmianę danych

In [ ]:
class ItemAdapter(itemComparator: ItemComparator) 
    : ListAdapter<Item, ItemViewHolder>(itemComparator) {

    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): ItemViewHolder {
        return ItemViewHolder(ItemRecyclerviewBinding.inflate(
            LayoutInflater.from(parent.context), parent, false
        ))
    }

    override fun onBindViewHolder(holder: ItemViewHolder, position: Int) {
        val item = getItem(position)
        holder.bind(item)
    }

    public fun getItemAt(position: Int): Item{
        return getItem(position)
    }
}

Oprócz metod które musimy nadpisać, dodajemy metodę publiczną `getItemAt` zwracającą element na zadanej pozycji - wykorzystamy ją przy implementacji przeszukiwania w `ListFragment`.

W metodzie `onViewCreated` klasy `Listfragment` dodajmy `RecyclerView`

In [ ]:
val adapter = ItemAdapter(ItemComparator())
binding.listRecyclerView.apply{
    adapter = adapter
    layoutManager = LinearLayoutManager(requireContext())
}

Następnie dodajmy obserwator

In [ ]:
itemViewModel.readAllData.observe(viewLifecycleOwner, adapter::submitList)

Dodajmy obsługę `onClick` dwóch `FAB` służących dodaniu i usunięciu wszystkich elementów

In [ ]:
binding.addItemFAB.setOnClickListener {
    findNavController()
        .navigate(ListFragmentDirections.actionListFragmentToAddFragment())}

binding.clearDataFAB.setOnClickListener { deleteAll()}

#### **swipeToDelete**

zaimplementujmy funkcję `swipeToDelete` - będziemy usuwać elementu z bazy poprzez wykonanie zdarzenia `swipe` w lewo lub prawo. Jako argument metoda przyjmuje `ItemAdapter`. W pierwszym kroku tworzymy `ItemTouchHelper` - argumentem jest `SimpleCallback`. Jako `dragDirs` podajemy 0 (nie będziemy wykorzystywać tej funkcjonalności), a jako `swipeDirs` podajemy odpowiednie `int` zdefiniowane w klasie `ItemTouchHelper`

In [ ]:
 private fun swipeToDelete(adapter: ItemAdapter) {
        ItemTouchHelper(object : ItemTouchHelper.SimpleCallback(
            0,
            ItemTouchHelper.RIGHT or ItemTouchHelper.LEFT
        ) {

Implementacja metody `onMove` nas nie interesuje, więc zwracamy `false`

In [ ]:
    override fun onMove(
        recyclerView: RecyclerView,
        viewHolder: RecyclerView.ViewHolder,
        target: RecyclerView.ViewHolder
    ): Boolean {
        return false
    }

W metodzie `onSwiped` wywołujemy `delete` klasy `ItemViewModel` - tutaj jako argument musimy podać element który chcemy usunąć, a mamy dostępną tylko pozycję elementu (przez metodę `getAdapterPosition`). W celu wyciągnięcia elementu wykorzystamy wcześniej zdefiniowaną metodę `getItemAt` w klasie `ItemAdapter`

In [ ]:
    override fun onSwiped(viewHolder: RecyclerView.ViewHolder, direction: Int) {
        itemViewModel.deleteItem(adapter.getItemAt(viewHolder.adapterPosition))
    }
}).attachToRecyclerView(binding.listRecyclerView)

Metodę `swipeToDelete` wywołujemy w `onViewCreated`

In [ ]:
swipeToDelete(adapter)

#### **searchView**

Ostatnim elementem tej aplikacji jest implementacja `SearchView`, pozwalająca na przeszukanie bazy. W tym celu stwórzmy nową metodę `setupSearchView` - również tutaj jako parametr przekażemy `ItemAdapter`

In [ ]:
private fun setupSearchView(adapter: ItemAdapter) {

Tutaj dodamy `setOnQueryTextListener` do naszego `SearchView`, który wymaga implementacji dwóch metod
- `onQueryTextSubmit` - wykonana po zakończeniu edycji i zatwierdzeniu
- `onQueryTextChange` - wykonana przy każdej zmianie tekstu w polu `SearchView`

W obu tych metodach wykonamy tą samą metodę - `search`.

In [ ]:
    binding.searchSearchView
    .setOnQueryTextListener(object : SearchView.OnQueryTextListener {
        override fun onQueryTextSubmit(query: String?): Boolean {
            if (query != null) search(query, adapter)
            return true
        }

        override fun onQueryTextChange(newText: String?): Boolean {
            if (newText != null) search(newText, adapter)
            return true
        }

    })

Zdefiniujmy metodę `search` - argumentem jest tekst do wyszukania oraz `ItemAdapter`, w pierwszym kroku zdefiniujmy `String` reprezentujący frazę do wyszukania

In [ ]:
private fun search(query: String, adapter: ItemAdapter){
    val searchQuery = "%$query%"

Nastęnie na `ItemViewModel` wykonajmy metodę `search` zwracającą listę wszystkich elementów pasujących do frazy wyszukania, oraz dodajmy obserwator i wykonajmy `submitList` - zaowocuje to natychmiastowym wyświetleniem wyniku w `RecyclerView`.

In [ ]:
    itemViewModel.searchItem(searchQuery)
    .observe(viewLifecycleOwner, adapter::submitList)

Możemy przetestować aplikację